In [137]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import *


In [226]:
import pandas as pd
import numpy as np
def load_dataset(path):
 df=pd.read_csv(path)
 
 assert df.isna().any().any()==False, 'There is at least one missing value.'
 assert df['Timestamp'].is_monotonic_increasing, 'Timestamp is not sorted.'
 
 df['abstime']=pd.to_datetime(df['Timestamp'], unit='s').round('us')
 df['monotime']=df['Timestamp']-df['Timestamp'].min()
 df['aid_int']=df['Arbitration_ID'].map(lambda x: int(x,16))
 df['y']=df['Class'].map({'Normal': 0, 'Attack': 1})
 
 df['time_interval']=df.groupby('Arbitration_ID')['Timestamp'].diff()
 
 return df
dfx=[]
for i in range(0,3):
    df_stationary_path="./0_Preliminary/0_Training/Pre_train_D_{t}.csv".format(t=i)
    dfx.append(load_dataset(df_stationary_path))



df_stationary=pd.concat([dfx[0],dfx[1],dfx[2]],ignore_index=True)

print(df_stationary)

def get_H(series_aid):
 count=series_aid.value_counts()
 p_i=count/series_aid.shape[0]
 return -(p_i*np.log(p_i)).sum()

df_stationary['entropy']=df_stationary.rolling(window=2402,min_periods=2402,step=10)['aid_int'].apply(get_H)
df_stationary['entropy']=df_stationary['entropy'].ffill()

df_stationary_except_nan=df_stationary[(~df_stationary['time_interval'].isna())&(~df_stationary['entropy'].isna())]

print(df_stationary_except_nan)



            Timestamp Arbitration_ID  DLC                     Data   Class  \
0        1.597708e+09            260    8  06 25 05 30 FF CF 71 55  Normal   
1        1.597708e+09            329    8  4A C5 7E 8C 31 2D 01 10  Normal   
2        1.597708e+09            38D    8  00 00 49 00 90 7F FE 01  Normal   
3        1.597708e+09            420    8  50 1E 00 C8 FC 4F 6A 00  Normal   
4        1.597708e+09            421    8  FE 07 00 FF E3 7F 00 52  Normal   
...               ...            ...  ...                      ...     ...   
1875126  1.599047e+09            153    8  20 A1 10 FF 00 FF 20 EF  Normal   
1875127  1.599047e+09            220    8  10 24 7F 74 03 FF BF 71  Normal   
1875128  1.599047e+09            329    8  D6 BF 7E 8C 32 2B 00 10  Normal   
1875129  1.599047e+09            47F    8  04 E0 FF FF 00 7B 00 57  Normal   
1875130  1.599047e+09            48A    8  00 00 07 41 03 02 60 88  Normal   

                           abstime    monotime  aid_int  y  tim

In [227]:
print((df_stationary_except_nan['Data'].iloc[0]))
datafield=[[int(i,16) for i in lst.split(' ')]+[-1]*(8-len(lst.split(' '))) for lst in df_stationary_except_nan['Data']]

df_df=pd.DataFrame(datafield,columns=['datafield{i}'.format(i=z) for z in range(0,8)])
#print(df_df)
df_1=df_stationary_except_nan.reset_index()
df_df=df_df.reset_index()
df_final=pd.concat([df_1,df_df],axis=1)

#df_final_y=df_final[df_final["y"]==1]
#df_final.drop(df_final[df_final["y"]==1].index,axis=0,inplace=True)
#print(df_final)

features_stationary=df_final[['aid_int','time_interval','entropy','datafield0','datafield1','datafield2','datafield3','datafield4','datafield5','datafield6','datafield7']]
labels_stationary=df_final['y']

from sklearn.model_selection import train_test_split

#data_stationary_train, data_stationary_test, target_stationary_train, target_stationary_test=train_test_split(features_stationary, labels_stationary, test_size=0.10, random_state=7)


00 00 00 24 76 01 45 30


In [234]:
print(len(features_stationary),len(labels_stationary))

1872558 1872558


In [230]:
df_test=load_dataset("./0_Preliminary/1_Submission/Pre_submit_D.csv")
df_test['entropy']=df_test.rolling(window=2402,min_periods=2402,step=10)['aid_int'].apply(get_H)
df_test['entropy']=df_test['entropy'].ffill()

df_test_except_nan=df_test[(~df_test['time_interval'].isna())&(~df_test['entropy'].isna())]

print(df_test_except_nan)

            Timestamp Arbitration_ID  DLC                     Data   Class  \
2410     1.599047e+09            386    8  00 00 00 C0 00 40 00 80  Normal   
2411     1.599047e+09            356    8  00 00 00 80 1A 00 00 00  Normal   
2412     1.599047e+09            366    7     25 EC 0A 25 23 00 01  Normal   
2413     1.599047e+09            367    8  00 00 00 00 00 00 C8 0A  Normal   
2414     1.599047e+09            368    8  00 00 00 00 00 EC 0A 40  Normal   
...               ...            ...  ...                      ...     ...   
2000728  1.599048e+09            453    5           00 88 8D 00 E2  Normal   
2000729  1.599048e+09            164    4              00 08 09 5E  Normal   
2000730  1.599048e+09            394    8  00 C0 08 00 D8 53 96 FC  Normal   
2000731  1.599048e+09            490    8  00 00 08 21 00 F0 3C 9E  Normal   
2000732  1.599048e+09            387    8  C1 AA F8 50 00 B6 03 00  Normal   

        SubClass                    abstime    monotime  aid_in

In [231]:
datafieldt=[[int(i,16) for i in lst.split(' ')]+[-1]*(8-len(lst.split(' '))) for lst in df_test_except_nan['Data']]

df_dft=pd.DataFrame(datafieldt,columns=['datafield{i}'.format(i=z) for z in range(0,8)])
#print(df_df)
df_1t=df_test_except_nan.reset_index()
df_dft=df_dft.reset_index()
df_final_test=pd.concat([df_1t,df_dft],axis=1)

#print(df_final)

features_stationary_t=df_final_test[['aid_int','time_interval','entropy','datafield0','datafield1','datafield2','datafield3','datafield4','datafield5','datafield6','datafield7']]
labels_stationary_t=df_final_test['y']

from sklearn.model_selection import train_test_split

#data_stationary_train, data_stationary_test, target_stationary_train, target_stationary_test=train_test_split(features_stationary, labels_stationary, test_size=0.10, random_state=7)


Index([ 266438,  266442,  266445,  266449,  266451,  266454,  266457,  266460,
        266462,  266464,
       ...
       1759248, 1759249, 1759253, 1759263, 1759265, 1759267, 1759271, 1759274,
       1759276, 1759279],
      dtype='int64', length=150485)
<class 'list'>


In [161]:
print((features_stationary),type(labels_stationary),type(features_stationary_t),type(labels_stationary_t))

         aid_int  time_interval   entropy  datafield0  datafield1  datafield2  \
0            832       0.010204  3.705266           0           0           0   
1            905       0.019995  3.705266           0           0           0   
2           1151       0.019994  3.705266           0          30         255   
3            897       0.019995  3.705266         128          64          60   
4            304       0.010009  3.705266         204         127          20   
...          ...            ...       ...         ...         ...         ...   
1872553      339       0.009985  3.704465          32         161          16   
1872554      544       0.009994  3.704465          16          36         127   
1872555      809       0.009623  3.704465         214         191         126   
1872556     1151       0.020106  3.704465           4         224         255   
1872557     1162       0.050029  3.705127           0           0           7   

         datafield3  datafi

In [49]:

fe_st=df_final[['aid_int','time_interval','entropy']]
la_st=df_final['y']
dstr,dste,tstr,tste=train_test_split(fe_st,la_st,test_size=0.10,random_state=7)

        aid_int  time_interval   entropy
392607      356       0.009784  3.707086
745156      608       0.009166  4.006286
186096      320       0.010046  3.685664
687930      854       0.010529  3.702993
659323      809       0.010055  3.704665
...         ...            ...       ...
759363     1988       0.040123  4.005056
535030      688       0.010243  3.716155
573977      304       0.010024  3.663474
189636     1173       0.100670  3.664708
585903      897       0.019394  3.704550

[798279 rows x 3 columns]


In [6]:
from sklearn.metrics import *
def evaluation(model, data_test, target_test):
 prediction=model.predict(data_test)
 
 print("{}:".format('Confusion matrix: \n'),confusion_matrix(target_test,prediction))
 print("{}:".format('Accuracy: \n'),accuracy_score(target_test,prediction))
 print("{}:".format('Precision: \n'),precision_score(target_test,prediction))
 print("{}:".format('Recall: \n'),recall_score(target_test,prediction))
 print("{}:".format('F1 score: \n'),f1_score(target_test,prediction))

data_stationary_train=np.array(data_stationary_train)
data_stationary_test=np.array(data_stationary_test)
target_stationary_train=np.array(target_stationary_train)
target_stationary_test=np.array(target_stationary_test)

In [ ]:
df=pd.read_csv("./0_Preliminary/0_Training/Pre_train_D_2.csv")
df

,Timestamp,Arbitration_ID,DLC,Data,Class,SubClass
0,1.599046e+09,164,4,00 08 12 5C,Normal,Normal
1,1.599046e+09,340,8,00 00 00 24 D6 01 B5 30,Normal,Normal
2,1.599046e+09,386,8,00 00 00 80 00 40 00 80,Normal,Normal
3,1.599046e+09,485,4,02 00 00 00,Normal,Normal
4,1.599046e+09,130,8,04 80 58 80 00 00 0E 56,Normal,Normal
...,...,...,...,...,...,...
889390,1.599047e+09,153,8,20 A1 10 FF 00 FF 20 EF,Normal,Normal
889391,1.599047e+09,220,8,10 24 7F 74 03 FF BF 71,Normal,Normal
889392,1.599047e+09,329,8,D6 BF 7E 8C 32 2B 00 10,Normal,Normal
889393,1.599047e+09,47F,8,04 E0 FF FF 00 7B 00 57,Normal,Normal


In [264]:
from tensorflow import keras
print(type(features_stationary.values))
X=features_stationary.values

X = X.reshape(X.shape[0], X.shape[1], 1)

model = keras.Sequential()
model.add(keras.layers.LSTM(32, input_shape=(11, 1),return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.LSTM(16))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()
model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0001),loss="binary_crossentropy",metrics=["accuracy"])

<class 'numpy.ndarray'>
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_34 (LSTM)              (None, 11, 32)            4352      
                                                                 
 dropout_24 (Dropout)        (None, 11, 32)            0         
                                                                 
 lstm_35 (LSTM)              (None, 16)                3136      
                                                                 
 dense_39 (Dense)            (None, 16)                272       
                                                                 
 dense_40 (Dense)            (None, 1)                 17        
                                                                 
Total params: 7777 (30.38 KB)
Trainable params: 7777 (30.38 KB)
Non-trainable params: 0 (0.00 Byte)
_____________________________________________________________

In [79]:
def train(model,data,target):
    model.fit(data,target,epochs=5)
    return model


In [262]:
model=train(model,X,labels_stationary.values)

Epoch 1/5
58518/58518 [==============================] - 188s 3ms/step - loss: 0.0841 - accuracy: 0.9804
Epoch 2/5
58518/58518 [==============================] - 188s 3ms/step - loss: 0.0918 - accuracy: 0.9766
Epoch 3/5
58518/58518 [==============================] - 179s 3ms/step - loss: 0.0883 - accuracy: 0.9787
Epoch 4/5
58518/58518 [==============================] - 177s 3ms/step - loss: 0.0876 - accuracy: 0.9807
Epoch 5/5
58518/58518 [==============================] - 174s 3ms/step - loss: 0.0893 - accuracy: 0.9794


In [263]:
#pred=model.predict()
def evaluations(model, data_test, target_test):
 predictions=model.predict(data_test)
 preds_1d = predictions.flatten() # 차원 펴주기
 prediction = np.where(preds_1d > 0.5, 1 , 0)
 
 print("{}:".format('DNN Confusion matrix: \n'),confusion_matrix(target_test,prediction))
 print("{}:".format('DNN Accuracy: '),accuracy_score(target_test,prediction))
 print("{}:".format('DNN Precision: '),precision_score(target_test,prediction))
 print("{}:".format('DNN Recall: '),recall_score(target_test,prediction))
 print("{}:".format('DNN F1 score: '),f1_score(target_test,prediction))
evaluations(model,features_stationary_t,labels_stationary_t)

62448/62448 [==============================] - 65s 1ms/step
DNN Confusion matrix: 
: [[1749163   47473]
 [  61815  139864]]
DNN Accuracy: : 0.9453099236106419
DNN Precision: : 0.7465903692276485
DNN Recall: : 0.6934980835882765
DNN F1 score: : 0.7190655397207313


In [247]:
model_dnn=keras.Sequential()

model_dnn.add(keras.layers.Dense(units=16,activation="relu",input_shape=(11,)))
model_dnn.add(keras.layers.Dense(units=16,activation="relu"))
model_dnn.add(keras.layers.Dense(units=1,activation="sigmoid"))



In [253]:
model_dnn.summary()
model_dnn.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 16)                192       
                                                                 
 dense_31 (Dense)            (None, 16)                272       
                                                                 
 dense_32 (Dense)            (None, 1)                 17        
                                                                 
Total params: 481 (1.88 KB)
Trainable params: 481 (1.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [254]:
model_dnn=train(model_dnn,features_stationary,labels_stationary)

Epoch 1/5
58518/58518 [==============================] - 25s 432us/step - loss: 0.0845 - accuracy: 0.9800
Epoch 2/5
58518/58518 [==============================] - 26s 441us/step - loss: 0.0782 - accuracy: 0.9823
Epoch 3/5
58518/58518 [==============================] - 25s 425us/step - loss: 0.0755 - accuracy: 0.9832
Epoch 4/5
58518/58518 [==============================] - 25s 426us/step - loss: 0.0740 - accuracy: 0.9837
Epoch 5/5
58518/58518 [==============================] - 25s 427us/step - loss: 0.0737 - accuracy: 0.9839


In [257]:
evaluations(model_s,features_stationary_t,labels_stationary_t)

62448/62448 [==============================] - 20s 317us/step
DNN Confusion matrix: 
: [[1768394   28242]
 [  62284  139395]]
DNN Accuracy: : 0.9546988337674491
DNN Precision: : 0.831528839098767
DNN Recall: : 0.6911726059728579
DNN F1 score: : 0.7548819980721116
